<div style="padding:24px; background-color:#f8f9fa; border:1px solid #dee2e6; border-radius:10px; line-height:1.6; color:#212529;">
    <h1 style="margin-top:0; color:#0d6efd;">🛍️ Case Study: Bon Prix Personalization Lab</h1>
    <hr style="border:0; height:1px; background:#dee2e6; margin:16px 0;">
    <p style="font-size:1.05em;">
        <b>Die Herausforderung:</b> Kunden im E-Commerce sind von tausenden Rezensionen überfordert.
        Ein reines Sterne-Rating bildet individuelle Bedürfnisse nicht ab.
    </p>
    <ul style="margin-left:20px;">
        <li><b>Ziel:</b> Personalisierte KI-Zusammenfassungen pro Kundin</li>
        <li><b>Technologie:</b> LLMs, NLP, Python & Gradio</li>
        <li><b>Business Value:</b> Höhere Conversion & weniger Retouren</li>
    </ul>
</div>

In [ ]:
# ==========================================
# 1. WERKZEUGKASTEN & SETUP
# ==========================================
import pandas as pd     # Datenanalyse (Das "Excel" von Python)
import gradio as gr     # Erstellung der Web-Oberfläche
import os, json, re     # Datei- und Textverarbeitung
import plotly.graph_objects as go  # Interaktive Business-Grafiken
from dotenv import load_dotenv
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# .env Datei laden (Damit wir unsere Keys nicht hardcoden müssen)
load_dotenv()


# ==========================================
# KONFIGURATION: Wähle eine der beiden Optionen
# ==========================================

# OPTION 1: Credentials aus .env Datei (EMPFOHLEN für Sicherheit)
# Stelle sicher, dass deine .env Datei korrekt ausgefüllt ist
USE_ENV_FILE = False

# OPTION 2: Credentials direkt hier eintragen (NUR für schnelle Tests)
# Setze USE_ENV_FILE = False und trage deine Werte unten ein
HARDCODED_API_KEY = ""  # Dein IBM Cloud API Key
HARDCODED_PROJECT_ID = ""  # Deine watsonx.ai Project ID
HARDCODED_URL = "https://us-south.ml.cloud.ibm.com"  # Standard für alle Studenten


# --- WATSONX SETUP MIT FEHLERBEHANDLUNG ---
if USE_ENV_FILE:
    IAM_API_KEY = os.getenv("WATSONX_API_KEY")
    WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")  # Fallback auf us-south
    PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")
    
    # Validierung der .env Werte
    if not IAM_API_KEY or IAM_API_KEY.strip() == "":
        raise ValueError(
            "❌ FEHLER: WATSONX_API_KEY ist nicht gesetzt!\n"
            "Lösung 1: Fülle die .env Datei korrekt aus\n"
            "Lösung 2: Setze USE_ENV_FILE = False und trage die Werte direkt im Code ein"
        )
    if not PROJECT_ID or PROJECT_ID.strip() == "":
        raise ValueError(
            "❌ FEHLER: WATSONX_PROJECT_ID ist nicht gesetzt!\n"
            "Lösung 1: Fülle die .env Datei korrekt aus\n"
            "Lösung 2: Setze USE_ENV_FILE = False und trage die Werte direkt im Code ein"
        )
    print("✅ Credentials erfolgreich aus .env geladen")
else:
    IAM_API_KEY = HARDCODED_API_KEY
    WATSONX_URL = HARDCODED_URL
    PROJECT_ID = HARDCODED_PROJECT_ID
    
    # Validierung der hardcoded Werte
    if not IAM_API_KEY or IAM_API_KEY.strip() == "":
        raise ValueError(
            "❌ FEHLER: HARDCODED_API_KEY ist leer!\n"
            "Trage deinen IBM Cloud API Key in HARDCODED_API_KEY ein"
        )
    if not PROJECT_ID or PROJECT_ID.strip() == "":
        raise ValueError(
            "❌ FEHLER: HARDCODED_PROJECT_ID ist leer!\n"
            "Trage deine watsonx.ai Project ID in HARDCODED_PROJECT_ID ein"
        )
    print("✅ Hardcoded Credentials werden verwendet")

# URL Validierung und Fallback
if not WATSONX_URL or WATSONX_URL.strip() == "":
    WATSONX_URL = "https://us-south.ml.cloud.ibm.com"
    print("⚠️  Keine URL angegeben, verwende Standard: us-south")

print(f"🔗 Verwende watsonx URL: {WATSONX_URL}")
print(f"📦 Projekt ID: {PROJECT_ID[:8]}...")


# 1. Parameter definieren (Was bedeutet das eigentlich? => https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-parameters.html?context=wx)
generate_params = {
    GenParams.MAX_NEW_TOKENS: 400,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1.1
}

# 2. ModelInference Instanz erstellen
try:
    model_inference = ModelInference(
        model_id="ibm/granite-3-3-8b-instruct",  # Granite 3.0 8B Instruct Modell
        params=generate_params,
        credentials=Credentials(api_key=IAM_API_KEY, url=WATSONX_URL),
        project_id=PROJECT_ID
    )
    print("✅ watsonx.ai ModelInference erfolgreich initialisiert")
except Exception as e:
    print(f"❌ FEHLER beim Initialisieren von watsonx.ai: {e}")
    print("\nMögliche Ursachen:")
    print("1. API Key ist ungültig oder abgelaufen")
    print("2. Project ID ist falsch")
    print("3. Keine Internetverbindung")
    print("4. watsonx.ai Service ist nicht verfügbar")
    raise


# Dateipfade (Unsere Datenbasis)
USER_FILE = 'personalization_users_visible.csv' 
REVIEWS_FILE = 'reviews_all_users_in_shop.csv'
CACHE_FILE = "bonprix_project_cache.json"

# Cache-Funktionen (Damit wir nicht für jede Anfrage die KI neu befragen - und bezahlen - müssen)
def get_cache():
    if not os.path.exists(CACHE_FILE): return {}
    try:
        with open(CACHE_FILE, "r", encoding="utf-8") as f: return json.load(f)
    except: return {}

def set_cache(key, val):
    c = get_cache()
    c[key] = val
    with open(CACHE_FILE, "w", encoding="utf-8") as f: 
        json.dump(c, f, indent=4, ensure_ascii=False)

<div style="padding:18px; background-color:#f5f7fa; border-left:4px solid #0d6efd; border-radius:6px; color:#212529;">
    <h3 style="margin-top:0;">📊 Schritt 1: Das „Gedächtnis“ der KI</h3>
    <p>
        Bevor die KI beraten kann, muss sie verstehen, <i>mit wem</i> sie spricht.
        Dafür kombinieren wir zwei Datenquellen:
    </p>
    <table style="width:100%; border-collapse:collapse; margin-top:10px;">
        <tr>
            <td style="padding:8px; border:1px solid #dee2e6;"><b>Kundenprofile</b></td>
            <td style="padding:8px; border:1px solid #dee2e6;">Historische Rezensionen & Präferenzen</td>
        </tr>
        <tr>
            <td style="padding:8px; border:1px solid #dee2e6;"><b>Produktrezensionen</b></td>
            <td style="padding:8px; border:1px solid #dee2e6;">Aktuelles Feedback anderer Kunden</td>
        </tr>
    </table>
</div>

In [ ]:
class BonPrixEngine:
    def __init__(self):
        # 1. Alle Shop-Rezensionen laden
        self.reviews = pd.read_csv(REVIEWS_FILE, sep=';')
        self.reviews.columns = [c.strip() for c in self.reviews.columns]
        
        # 2. User-Profile & Historie aufbauen
        self.users = {}
        if os.path.exists(USER_FILE):
            df_u = pd.read_csv(USER_FILE, sep=';')
            df_u.columns = [c.strip() for c in df_u.columns]
            
            for name in df_u['user_name'].unique():
                if pd.isna(name): continue
                sub = df_u[df_u['user_name'] == name]
                
                # Wir verknüpfen alle alten Rezensionstexte zu einem "Profil-String"
                past_reviews = " | ".join(sub['review_text'].dropna().astype(str).tolist())
                
                # Einfache statistische Merkmale (z.B. Durchschnittsgröße)
                sz_raw = sub['size'].mode()[0] if not sub['size'].mode().empty else "40"
                sz = int(re.search(r'\d+', str(sz_raw)).group()) if re.search(r'\d+', str(sz_raw)) else 38
                
                self.users[name] = {
                    "base_size": sz,
                    "height": sub['body_size'].mode()[0] if not sub['body_size'].mode().empty else "Unbekannt",
                    "avg_rating": round(sub['rating'].mean(), 1) if not sub['rating'].empty else "N/A",
                    "past_reviews": past_reviews 
                }
        
        # 3. Produktkatalog (Top 20 Produkte für die Demo)
        top_p = self.reviews['product_id'].value_counts().head(20).index.tolist()
        self.products = self.reviews[self.reviews['product_id'].isin(top_p)][['product_id', 'short_description']]\
            .drop_duplicates().set_index('product_id')['short_description'].to_dict()

engine = BonPrixEngine()

<div style="padding:18px; background-color:#f8f9fa; border-left:4px solid #6c757d; border-radius:6px; color:#212529;">
    <h3 style="color:#2c3e50; margin-top:0;">🧠 Schritt 2: Prompt Engineering (Relevanz-Steuerung)</h3>
    <p>
        Hier definieren wir das <b>Business-Briefing</b>. Als Wirtschaftsinformatiker steuern Sie die KI nicht durch starre Programmierung, sondern durch <b>adaptive Instruktionen</b>.
    </p>
    <blockquote style="margin:12px 0; padding-left:12px; border-left:3px solid #adb5bd; color:#495057; font-style:italic;">
        "Erkenne das Kundenbedürfnis aus der Historie. Wenn die Kundin Passform-Fragen ignoriert, soll die KI das auch tun. Wir liefern Information, keinen Ballast."
    </blockquote>
    <p style="font-size:0.95em;">
        <b>Challenge:</b> Wir nutzen die KI hier für die <i>Interpretation</i> von Nuancen, während wir harte Fakten (wie Größenempfehlungen) den spezialisierten Systemen überlassen.
    </p>
</div>

In [ ]:
def build_student_prompt(user_name, profile, product_name, current_product_reviews):
    """
    Diese Funktion baut das 'Briefing' für die KI.
    Hier fließt das Business-Wissen ein (Worauf soll die KI achten?).
    """
    
    user_meta = f"Körpergröße: {profile.get('height')}, Ø-Rating: {profile.get('avg_rating')}"
    
    return f"""
    Die Kundin {user_name} ({user_meta}) schaut sich gerade das Produkt "{product_name}" an.
    
    REZENSIONEN ZUM AKTUELLEN PRODUKT:
    {current_product_reviews}
    
    HISTORIE DER KUNDIN (IHRE ALTEN REZENSIONEN):
    "{profile['past_reviews']}"
    
    AUFGABE:
    Erstelle eine personalisierte Zusammenfassung der Rezensionen zum neuen Produkt für {user_name}.
    
    INSTRUKTIONEN:
    1. ANALYSE: Welche Aspekte (z.B. Material, Stil, Nachhaltigkeit) sind für diese spezifische Kundin 
       laut ihren alten Rezensionen von Bedeutung?
    2. PERSONALISIERUNG: Fasse die neuen Rezensionen NUR im Hinblick auf diese persönlichen Prioritäten zusammen.
    3. PASSFORM-VETO: Gehe NUR auf die Passform ein, wenn die Kundin diesen Punkt in ihrer 
       Vergangenheit selbst erwähnt hat.
    4. KONTEXT: Berücksichtige Angaben anderer Kundinnen (z.B. "fällt lang aus") nur, wenn sie 
       im Hinblick auf die Metadaten der Kundin ({user_meta}) relevant sind.
    
    WICHTIG: Max. 4 Sätze, ehrlich, direkt, Deutsch.
    """

In [ ]:
# ==========================================
# 3. ANALYSE-LOGIK (Berechnungen & KI-Aufruf)
# ==========================================

def run_analysis(user_name, product_selection):
    if not user_name or not product_selection: 
        return [None]*3 + ["### ⚠️ Bitte links Kundin und Produkt wählen!", ""]
    
    pid = int(product_selection.split(' - ')[0])
    product_name = engine.products.get(pid, "Unbekanntes Produkt")
    profile = engine.users[user_name]
    df = engine.reviews[engine.reviews['product_id'] == pid]
    
    # --- A. VISUALISIERUNG: Passform & Aspekte ---
    txt_all = " ".join(df['review_text'].dropna().astype(str).tolist()).lower()
    
    # Passform-Tacho
    s, l = len(re.findall(r'klein|eng|kurz', txt_all)), len(re.findall(r'groß|weit|lang', txt_all))
    fit_score = (l - s) / (l + s + 1)
    fig_gauge = go.Figure(go.Indicator(mode="gauge+number", value=fit_score, title={'text': "Passform-Tendenz"},
                                       gauge={'axis': {'range': [-1, 1]}, 'bar': {'color': "#c5003d"}}))
    
    # Radar-Chart für Produkt-Eigenschaften
    def get_s(kw): return min(100, int((sum(txt_all.count(w) for w in kw) / (len(df)+1)) * 300))
    aspects = {"Komfort": get_s(["bequem", "weich"]), "Qualität": get_s(["qualität", "stoff"]), 
               "Stil": get_s(["schön", "optik"]), "Preis": get_s(["preis", "günstig"])}
    fig_radar = go.Figure(data=go.Scatterpolar(r=list(aspects.values()), theta=list(aspects.keys()), fill='toself'))
    
    # Sterne-Verteilung
    counts = df['rating'].value_counts().sort_index()
    fig_stars = go.Figure(data=[go.Bar(x=[f"{i}★" for i in counts.index], y=counts.values, marker_color='#f1c40f')])

    for f in [fig_gauge, fig_radar, fig_stars]: 
        f.update_layout(height=280, margin=dict(l=40, r=40, t=40, b=40), template="plotly_white")

    # --- B. KI-PERSONALISIERUNG ---
    reviews_sample = "\n- ".join(df['review_text'].head(15).astype(str).tolist())[:1500]
    final_prompt = build_student_prompt(user_name, profile, product_name, reviews_sample)
    
    cache = get_cache()
    if f"{user_name}_{pid}" in cache:
        return fig_gauge, fig_radar, fig_stars, cache[f"{user_name}_{pid}"], final_prompt
    else:
        try:
            generated_text = model_inference.generate_text(prompt=final_prompt)

            set_cache(f"{user_name}_{pid}", generated_text)
            
        except Exception as e:
            generated_text = f"### ⚠️ KI-Fehler\nWatsonX ist nicht gestartet: {e}"
    
    return fig_gauge, fig_radar, fig_stars, generated_text, final_prompt
            

<div style="padding:18px; background-color:#f5f7fa; border-left:4px solid #198754; border-radius:6px; color:#212529;">
    <h3 style="margin-top:0;">🚀 Schritt 3: Das MVP (Minimum Viable Product)</h3>
    <p>
        Ein Algorithmus ohne Interface ist für den Fachbereich nicht nutzbar.
        Mit <b>Gradio</b> bauen wir eine Web-Oberfläche, die den Business-Mehrwert sichtbar macht.
    </p>
    <p><b>Starten Sie unten das Labor, um das MVP live zu testen.</b></p>
</div>

In [ ]:
# UI DESIGN
with gr.Blocks() as demo:
    gr.Markdown("# 🛍️ Bon Prix: Personalization Lab")
    
    with gr.Row():
        with gr.Column(scale=1, variant="panel"):
            gr.Markdown("### 🛠️ Konfiguration")
            user_dd = gr.Dropdown(choices=list(engine.users.keys()), value=list(engine.users.keys())[0] if engine.users else None, label="1. Kundin wählen")
            prod_dd = gr.Dropdown(choices=[f"{k} - {v}" for k,v in engine.products.items()], label="2. Produkt wählen")
            btn = gr.Button("🚀 Personalisierte Analyse", variant="primary")
            gr.Markdown("---")
            gr.Markdown("**Info:** Die KI analysiert im Hintergrund die Profile von echten bon prix Kundinnen.")

        with gr.Column(scale=2):
            with gr.Tabs():
                with gr.TabItem("📝 KI-Beratung"):
                    ai_out = gr.Markdown("### 💡 Ergebnis\nWählen Sie links eine Kundin und ein Produkt...")
                    with gr.Accordion("🔍 Prompt-Einsicht (Technik)", open=False):
                        prompt_out = gr.Code(language="markdown")
                
                with gr.TabItem("📊 Statistiken"):
                    with gr.Row():
                        plot_gauge = gr.Plot()
                        plot_radar = gr.Plot()
                    plot_stars = gr.Plot()

    btn.click(run_analysis, inputs=[user_dd, prod_dd], outputs=[plot_gauge, plot_radar, plot_stars, ai_out, prompt_out])

# Start der App
if __name__ == "__main__":
    demo.launch(theme=gr.themes.Soft())